In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-02-02 00:03:52.053915: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 00:03:52.143735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))



In [3]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    #x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(inputs)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, activation='relu')(x)
    z_logvar=layers.Dense(latent_dim, activation='relu')(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder


#from v2 to v3 I removed the activation function from the last layer
def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder


#VAE Model Version 3
class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=0.5

    
    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        
        MSE_loss=tf.reduce_mean(keras.losses.mse(mask*data, mask*reconstruction))
        
        loss=self.beta*kl_loss+(1-self.beta)*MSE_loss
        
        self.total_loss_tracker.update_state(loss)
        self.reconstruction_loss_tracker.update_state((1-self.beta)*MSE_loss)
        self.kl_loss_tracker.update_state(self.beta*kl_loss)
        
        self.add_loss(loss)
        

        
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }
    
    def set_beta(self,beta):
        self.beta=beta
    


        
    

In [7]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_encoder(57,32,16,3)
vae_dec=make_decoder(57,32,16,3)
vae_40MHZ=VAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam()
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 1024

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_data=(Data_Val_Flat,Data_Val_Flat),epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/non_normed_new_beta_{}_v3/'.format(beta),save_format='tf')

Epoch 1/100
1954/1954 [==============================] - 11s 5ms/step - loss: 449278.6562 - total_loss: 449278.6562 - reconstruction_loss: 6112.7173 - kl_loss: 443169.5938 - val_loss: 2.3817 - val_total_loss: 2.3817 - val_reconstruction_loss: 2.0823 - val_kl_loss: 0.2994 - lr: 0.0010
Epoch 2/100
1954/1954 [==============================] - 9s 4ms/step - loss: 2.2585 - total_loss: 2.2585 - reconstruction_loss: 1.9717 - kl_loss: 0.2868 - val_loss: 2.2354 - val_total_loss: 2.2354 - val_reconstruction_loss: 1.9534 - val_kl_loss: 0.2821 - lr: 0.0010
Epoch 3/100
1954/1954 [==============================] - 9s 4ms/step - loss: 2.1935 - total_loss: 2.1935 - reconstruction_loss: 1.9135 - kl_loss: 0.2800 - val_loss: 2.1940 - val_total_loss: 2.1940 - val_reconstruction_loss: 1.9233 - val_kl_loss: 0.2707 - lr: 0.0010
Epoch 4/100
1954/1954 [==============================] - 8s 4ms/step - loss: 2.1671 - total_loss: 2.1671 - reconstruction_loss: 1.8876 - kl_loss: 0.2795 - val_loss: 2.1792 - val_total

Epoch 32/100
1954/1954 [==============================] - 8s 4ms/step - loss: 2.0693 - total_loss: 2.0693 - reconstruction_loss: 1.7514 - kl_loss: 0.3179 - val_loss: 2.0859 - val_total_loss: 2.0859 - val_reconstruction_loss: 1.7815 - val_kl_loss: 0.3044 - lr: 0.0010
Epoch 33/100
1954/1954 [==============================] - 8s 4ms/step - loss: 2.0686 - total_loss: 2.0686 - reconstruction_loss: 1.7510 - kl_loss: 0.3176 - val_loss: 2.0834 - val_total_loss: 2.0834 - val_reconstruction_loss: 1.7509 - val_kl_loss: 0.3325 - lr: 0.0010
Epoch 34/100
1954/1954 [==============================] - 8s 4ms/step - loss: 2.0672 - total_loss: 2.0672 - reconstruction_loss: 1.7485 - kl_loss: 0.3186 - val_loss: 2.0791 - val_total_loss: 2.0791 - val_reconstruction_loss: 1.7680 - val_kl_loss: 0.3112 - lr: 0.0010
Epoch 35/100
1954/1954 [==============================] - 8s 4ms/step - loss: 2.0674 - total_loss: 2.0674 - reconstruction_loss: 1.7488 - kl_loss: 0.3186 - val_loss: 2.0838 - val_total_loss: 2.0838 -

In [8]:
a=Data_Train_Flat[0:100]
b=vae_40MHZ.encoder(a)
print(b)

[<tf.Tensor: shape=(100, 3), dtype=float32, numpy=
array([[0.        , 0.        , 0.        ],
       [0.        , 3.1305077 , 0.        ],
       [0.        , 2.2455888 , 0.        ],
       [0.        , 3.0734463 , 0.        ],
       [0.        , 1.3216188 , 0.        ],
       [0.        , 0.60316646, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.98985153, 0.        ],
       [0.        , 0.566447  , 0.        ],
       [0.        , 0.04169333, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 1.2586201 , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 2.9763327 , 0.        ],
       [0.        , 1.8671203 , 0.        ],
       [0.        , 0.34932318, 0.        ],
       [0.        , 1.407789  , 0.        ],
    